# Calculating Mean Average Precision from yolo format txt files

In [8]:
import glob,os
import numpy as np
import supervision as sv
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category = FutureWarning)
pd.set_option('mode.chained_assignment', None)

In [9]:
def box_iou_batch(
	boxes_a: np.ndarray, boxes_b: np.ndarray
) -> np.ndarray:

    def box_area(box):
        return (box[2] - box[0]) * (box[3] - box[1])

    area_a = box_area(boxes_a.T)
    area_b = box_area(boxes_b.T)

    top_left = np.maximum(boxes_a[:, None, :2], boxes_b[:, :2])
    bottom_right = np.minimum(boxes_a[:, None, 2:], boxes_b[:, 2:])

    area_inter = np.prod(
    	np.clip(bottom_right - top_left, a_min=0, a_max=None), 2)
        
    return area_inter / (area_a[:, None] + area_b - area_inter)


def non_max_suppression(
   predictions: np.ndarray, iou_threshold: float = 0.5
) -> np.ndarray:
    rows, columns = predictions.shape
    sort_index = np.flip(predictions[:, 5].argsort())
    predictions = predictions[sort_index]
    boxes = predictions[:, :4]
    categories = predictions[:, 4]
    ious = box_iou_batch(boxes, boxes)
    ious = ious - np.eye(rows)

    keep = np.ones(rows, dtype=bool)

    for index, (iou, category) in enumerate(zip(ious, categories)):
        if not keep[index]:
            continue

        condition = (iou > iou_threshold) & (categories == category)
        keep = keep & ~condition

    return keep[sort_index.argsort()]



def xyxy(arr, type = None):
    # arr -- numpy array (n x 6)
    tmp = []
    for x in arr:
        actual_w = x[3] * 1120
        actual_h = x[4] * 1120
        actual_x_c = x[1] * 1120
        actual_y_c = x[2] * 1120
        xmin , ymin =  actual_x_c - (actual_w/2), actual_y_c - (actual_h/2)
        xmax, ymax = actual_x_c + (actual_w/2), actual_y_c + (actual_h/2)
        if type == 'P':
            tmp.append([xmin, ymin, xmax, ymax, x[0], x[-1]]) # xmin, xmax, ymin,ymax, class, conf
        else:
            tmp.append([xmin, ymin, xmax, ymax, x[0]])

    return np.array(tmp)


df = pd.DataFrame(columns = ['map50','map75','map50-95'], index = ['overall', 'FCBK', 'Zigzag'])    

In [ ]:
a = []
nms = False
num_folds = 4
for fold in range(num_folds):
    gt_dir =  f'/user/delhi_sarath_grid_aa_v1/{fold}/test/labels'
    pred_dir = f'/user/Fast_RCNN/crossval/high_res_v1_Delhi/{fold}'

    gt_files = glob.glob(os.path.join(gt_dir,'*.txt'))
    pred_files = glob.glob(os.path.join(pred_dir,'*.txt'))

    gt_results = []
    pred_files = []
    pred_results = []
    info = False

    for i in range(len(gt_files)):
        # gt_results.append(xyxy(np.loadtxt(gt_files[i], ndmin= 2)))
        file_name = os.path.basename(gt_files[i])
        corresponding_pred_file = os.path.join(pred_dir, file_name)
        if os.path.exists(corresponding_pred_file):
            pred_files.append(corresponding_pred_file)
            gt_results.append(xyxy(np.loadtxt(gt_files[i], ndmin= 2)))

    print(pred_files)
    if nms:
        for i in range(len(pred_files)):
            results = np.loadtxt(pred_files[i], ndmin= 2) # load all predictions of single image
            bool_list = non_max_suppression(xyxy(results, type = 'P')) #apply non max suppression
            best_boxes = []
            for i, element in enumerate(bool_list):
                if element:
                    best_boxes.append(results[i]) # add bbox with highest IOU and highest score
            pred_results.append(xyxy(best_boxes, type = 'P'))
            

    else:
        for i in range(len(pred_files)):
            if pred_files[i] == None:
                continue
            pred_results.append(xyxy(np.loadtxt(pred_files[i], ndmin= 2), type='P'))


    if info:
        print(f'No of GT images in Fold {fold}:', len(gt_results))
        print(f'No of bbox in one GT image in Fold {fold}', len(gt_results[0]), type(gt_results[0]))
        print('No of Pred images:', len(pred_results))
        print('No of predictions in one GT image', len(pred_results[0]), type(pred_results[0]))
        print('Shape of predictions in one GT image',pred_results[0].shape)

    #Finding mean average precision
    mean_average_precison = sv.MeanAveragePrecision.from_tensors(
                predictions= pred_results,
                targets= gt_results,
            )
    # print(mean_average_precison.per_class_ap50_95)
    map_fcbk , map_zigzag = mean_average_precison.per_class_ap50_95.mean(axis = 1)
    df = df.copy(deep = True)
    df.loc['overall']['map50'] = mean_average_precison.map50
    df.loc['overall']['map75'] = mean_average_precison.per_class_ap50_95[:,5].mean()
    df.loc['overall']['map50-95'] = mean_average_precison.map50_95
    df.loc['FCBK']['map50'] = mean_average_precison.per_class_ap50_95[:,0][0]
    df.loc['FCBK']['map75'] = mean_average_precison.per_class_ap50_95[:,5][0]
    df.loc['FCBK']['map50-95'] = map_fcbk
    df.loc['Zigzag']['map50'] = mean_average_precison.per_class_ap50_95[:,0][1]
    df.loc['Zigzag']['map75'] = mean_average_precison.per_class_ap50_95[:,5][1]
    df.loc['Zigzag']['map50-95'] = map_zigzag
    print(f'Fold {fold}')
    print(df,end = '\n\n')

    a.append(df)

In [ ]:
print('Combined MAP of all folds ')
(a[0]+a[1]+a[2]+a[3])/num_folds
mean_and_std = {}
maps = ["map50", "map75", "map50-95"]
for map in maps:
    mean_and_std[map] = {}
    for i in range(2):
        if i==0:
            type = 'FCBK'
        else:
            type = 'Zigzag'
        values = []
        for j in range(4):
            values.append(a[j].loc[type][map])
        mean_and_std[map][type] = (np.mean(values), np.std(values))


print(mean_and_std)

In [11]:
dir="/tmp/gujarat.csv"
gujarat_pred=pd.read_csv(dir)
gujarat_pred.head(5)

,x_min,y_min,x_max,y_max,class_id,confidence,tracker_id,base_name,class_name,xyxyxyxy
0,216.65506,1099.9314,255.30933,1119.8660,1,0.320954,NaN,"22.95,72.59.zarr",Zigzag,[[ 216.66 1100.1]\n [ 216.73 ...
1,969.08636,1105.6630,1096.67310,1121.2008,1,0.330701,NaN,"23.79,68.50.zarr",Zigzag,[[ 969.09 1106]\n [ 969.13 ...
2,553.57074,979.6705,632.88590,1048.8035,1,0.259546,NaN,"21.82,73.62.zarr",Zigzag,[[ 614.81 1048.8]\n [ 632.89 ...
3,198.81053,1091.5186,262.15787,1119.8276,1,0.392807,NaN,"23.25,70.26.zarr",Zigzag,[[ 262.08 1119.8]\n [ 262.16 ...
4,1108.05040,947.7308,1120.74160,1006.7807,1,0.305959,NaN,"21.98,70.65.zarr",Zigzag,[[ 1108.1 1006.7]\n [ 1120.1 ...


In [12]:
import pandas as pd
import numpy as np

def non_max_suppression(df, iou_threshold=0.5):
    # Convert bounding boxes from the DataFrame to numpy arrays
    boxes = df[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()
    confidences = df['confidence'].to_numpy()
    indices = np.arange(len(boxes))
    
    # Sort by confidence
    sorted_indices = np.argsort(confidences)[::-1]
    sorted_boxes = boxes[sorted_indices]
    sorted_confidences = confidences[sorted_indices]
    sorted_indices = indices[sorted_indices]

    selected_indices = []
    while len(sorted_indices) > 0:
        current_index = sorted_indices[0]
        selected_indices.append(current_index)
        
        if len(sorted_indices) == 1:
            break
        
        current_box = sorted_boxes[0]
        rest_boxes = sorted_boxes[1:]
        
        # Calculate IoU (Intersection over Union)
        x_min = np.maximum(current_box[0], rest_boxes[:, 0])
        y_min = np.maximum(current_box[1], rest_boxes[:, 1])
        x_max = np.minimum(current_box[2], rest_boxes[:, 2])
        y_max = np.minimum(current_box[3], rest_boxes[:, 3])

        inter_area = np.maximum(0, x_max - x_min) * np.maximum(0, y_max - y_min)
        box_area = (current_box[2] - current_box[0]) * (current_box[3] - current_box[1])
        rest_area = (rest_boxes[:, 2] - rest_boxes[:, 0]) * (rest_boxes[:, 3] - rest_boxes[:, 1])

        iou = inter_area / (box_area + rest_area - inter_area)
        
        # Filter out boxes with IoU above the threshold
        below_threshold_indices = np.where(iou < iou_threshold)[0]
        
        sorted_boxes = sorted_boxes[below_threshold_indices + 1]
        sorted_confidences = sorted_confidences[below_threshold_indices + 1]
        sorted_indices = sorted_indices[below_threshold_indices + 1]
    
    return df.iloc[selected_indices]


In [13]:
df = pd.DataFrame(gujarat_pred)
result_nms=non_max_suppression(df)
print(result_nms)



            x_min        y_min        x_max        y_max  class_id  \
542    670.398900   729.785160   773.812260   813.359400         0   
1050  1024.790000    93.735000  1132.491700   172.077960         0   
1644   791.790950   191.981810   903.609900   266.182620         0   
1925   766.835940   631.552060   896.087650   709.713560         0   
43    1012.587770   953.829400  1111.624400  1018.852700         0   
...           ...          ...          ...          ...       ...   
2121    80.418106    -0.373445   105.464160    24.410873         1   
2399   667.840330   487.704500   709.765000   565.415830         1   
1425  1026.487700  1081.409400  1092.773100  1128.194600         1   
940     -1.853052   629.599600    17.936623   697.648800         1   
1484   777.546600    -8.086359   871.288000    26.257082         1   

      confidence  tracker_id         base_name class_name  \
542     0.967423         NaN  22.12,72.82.zarr       FCBK   
1050    0.966807         NaN  23.14,7

In [21]:
class_counts = result_nms['class_id'].value_counts()
class_counts
num_class_1 = class_counts.get(1)
num_class_2 = class_counts.get(0)

print(f"Number of class zigzag: {num_class_1}")
print(f"Number of class fcbk: {num_class_2}")

Number of class zigzag: 1284
Number of class fcbk: 607


In [ ]:
import pandas as pd
import numpy as np

def iou(box1, box2):
    x_min = np.maximum(box1[0], box2[0])
    y_min = np.maximum(box1[1], box2[1])
    x_max = np.minimum(box1[2], box2[2])
    y_max = np.minimum(box1[3], box2[3])

    inter_area = np.maximum(0, x_max - x_min) * np.maximum(0, y_max - y_min)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    iou_value = inter_area / (box1_area + box2_area - inter_area)
    return iou_value

def non_max_suppression(df, iou_threshold=0.5):
    boxes = df[['x_min', 'y_min', 'x_max', 'y_max']].to_numpy()
    confidences = df['confidence'].to_numpy()
    indices = np.arange(len(boxes))

    sorted_indices = np.argsort(confidences)[::-1]
    sorted_boxes = boxes[sorted_indices]
    sorted_confidences = confidences[sorted_indices]
    sorted_indices = indices[sorted_indices]

    selected_indices = []
    while len(sorted_indices) > 0:
        current_index = sorted_indices[0]
        selected_indices.append(current_index)

        if len(sorted_indices) == 1:
            break

        current_box = sorted_boxes[0]
        rest_boxes = sorted_boxes[1:]

        ious = np.array([iou(current_box, box) for box in rest_boxes])
        below_threshold_indices = np.where(ious < iou_threshold)[0]

        sorted_boxes = sorted_boxes[below_threshold_indices + 1]
        sorted_confidences = sorted_confidences[below_threshold_indices + 1]
        sorted_indices = sorted_indices[below_threshold_indices + 1]

    return df.iloc[selected_indices]